## EDA FOR THE CLEANED DATASET

This brief notebook will perform some of the various EDA techniques used on the raw dataset in order to understand not only if the cleaned dataset is complete and suitable for further preprocessing-modeling steps, but also to see if the shape of the data changes importantly, or not. 

In [ ]:
file_path = r'C:\unibo-dtm-ml-2526-cervical-cancer-predictor\data\processed_data\cleaned_data_knn_imputation.csv'
with open(file_path, 'r') as f:
    lines = f.readlines()
print('Read {} lines'.format(len(lines)))

Importing the necessary libraries. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


Outlining the renewed profile of the dataset. 

In [ ]:
# repeat the data profiling pipeline for the newly cleaned data
df = pd.read_csv(file_path)

print("\nDataset Info: \n")
print(df.info())

#check whether everything went smoothly at the data cleaning stage
print("\nMissing Values: \n")
print(df.isnull().sum()) 

print("\nDuplicate Values: \n")
print(df.duplicated().sum())

print("\nDescriptive Statistics:")
print(df.describe(include='all'))
